In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on <09-16-2017>

This Dataset is from the Instacart Kaggle Competition. The challenge is to use this anonymized data on customer orders over time to predict which previously purchased products will be in a user’s next order. 

It was informed that this dataset is a relational set of 6 files describing customers' orders over time. 

We will start by reading all of them:

In [2]:
aisles = pd.read_csv('Instacart Kaggle/aisles.csv', index_col='aisle_id')
aisles.head()

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation


In [3]:
dptmts = pd.read_csv('Instacart Kaggle/departments.csv', index_col='department_id')
dptmts.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


In [4]:
prod_in_orders = pd.read_csv('Instacart Kaggle/order_products__prior.csv', dtype={'order_id': object, 'product_id': object})
prod_in_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
all_orders = pd.read_csv('Instacart Kaggle/orders.csv', dtype={'order_id':object, 'user_id':object, 'order_dow':object})
all_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
#train = pd.read_csv('Instacart Kaggle/order_products__train.csv')
train.head()

NameError: name 'train' is not defined

In [7]:
products = pd.read_csv('Instacart Kaggle/products.csv', index_col='product_id')
products.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [ ]:
#sample_sub = pd.read_csv('Instacart Kaggle/sample_submission.csv')
sample_sub.head()

That was smooth!!! 

Taking a quick look using .head() we can have an idea what the data is about.

Are the columns names consistents?
Are there any missing values?

aisles ok
dptmts ok
prod_in_orders ok
products ok
all_orders - days_since_prior: (with NAs for order_number = 1)

In [8]:
aisles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 1 to 134
Data columns (total 1 columns):
aisle    134 non-null object
dtypes: object(1)
memory usage: 2.1+ KB


In [9]:
aisles.describe()

,aisle
count,134
unique,134
top,fresh fruits
freq,1


In [10]:
dptmts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 1 to 21
Data columns (total 1 columns):
department    21 non-null object
dtypes: object(1)
memory usage: 336.0+ bytes


In [11]:
dptmts.describe()

,department
count,21
unique,21
top,canned goods
freq,1


In [12]:
prod_in_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             object
product_id           object
add_to_cart_order    int64
reordered            int64
dtypes: int64(2), object(2)
memory usage: 989.8+ MB


In [13]:
prod_in_orders.describe()

,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07
mean,8.351076e+00,5.896975e-01
std,7.126671e+00,4.918886e-01
min,1.000000e+00,0.000000e+00
25%,3.000000e+00,0.000000e+00
50%,6.000000e+00,1.000000e+00
75%,1.100000e+01,1.000000e+00
max,1.450000e+02,1.000000e+00


In [14]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 1 to 49688
Data columns (total 3 columns):
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [15]:
products.describe()

,aisle_id,department_id
count,49688.000000,49688.000000
mean,67.769582,11.728687
std,38.316162,5.850410
min,1.000000,1.000000
25%,35.000000,7.000000
50%,69.000000,13.000000
75%,100.000000,17.000000
max,134.000000,21.000000


In [16]:
all_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  object
user_id                   object
eval_set                  object
order_number              int64
order_dow                 object
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 182.7+ MB


In [17]:
all_orders.describe()

,order_number,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.715486e+01,1.345202e+01,1.111484e+01
std,1.773316e+01,4.226088e+00,9.206737e+00
min,1.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,1.000000e+01,4.000000e+00
50%,1.100000e+01,1.300000e+01,7.000000e+00
75%,2.300000e+01,1.600000e+01,1.500000e+01
max,1.000000e+02,2.300000e+01,3.000000e+01


In [ ]:
train.info()

In [ ]:
#here will look for outliers in the number of products per order
n_prod_per_order = prod_in_orders.groupby('order_id')['product_id'].count().sort_values()
n_prod_per_order.plot(kind='box')

we can see big numbers, but we can't conclude that they are not reasonable number for an order.
And the result matches the variable add_to_cart seen earlier if the .describe() function.

So, no outliers or missing values :-)))

Feature Engineering:

1- Drivers

I was disapointed when I looked into the variable reorder, as it just flags if it has been ordered before with 0 and 1. Intuitively, I assume that there are products that are more important to me in my cart than others. I would call them "drivers" because they are the ones that makes the order happen.
These drivers are more likelly to be one of the first itens in your cart and have been previously purchased.

a) First in the Cart
So we will create a column in prod_in_orders called drivers that is the first item included and is a reorder.

In [24]:
prod_in_orders['drivers'] = np.where(((prod_in_orders['add_to_cart_order'] == 1) & (prod_in_orders['reordered'] == 1)), 1, 0)
prod_in_orders.head()

,order_id,product_id,add_to_cart_order,reordered,drivers
0,2,33120,1,1,1
1,2,28985,2,1,0
2,2,9327,3,0,0
3,2,45918,4,1,0
4,2,30035,5,0,0


b) Users with Babies & Pets
As these Users have to look after the needs of their loved ones (either pets or babies) I would assume we can expect more of a routine in their grouciries shopping. In a more detailed work we should think of surrogate key, as babies speacilly grows and have different needs.
For now we will identify the base products in those aisles and/or departments and flag them as drivers.

c) Frequencies of drivers
Not everytime we have the same reason to shop, thats why the main product that made you do the grocieries can differ every now and again. Each product that we own runs out in different times, making different frequencies of drivers. For example my husband and I has this frequencies: coke daily, pasta weekly, shampoo monthly.
We will calculate how many drivers per user, then how many different times in the cart and how often in the cart.

2- Strong ties between products

Every user combine products as their personal preference. So we can imagine that some products may have strong ties , like the famous example of Diapers and Beer. The ambition here is to see if we can find more pairs or trios. 
For that we will create a matrix 2D product vs product and counts how many times the pair was in the cart together.